In [1]:
!pip install tensorflow==2.2.0

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 460kB 44.0MB/s 
     |████████████████████████████████| 3.0MB 55.2MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [8]:
import tensorflow as tf

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
#from tensorflow.keras.optimizers import Adam
from tensorflow import keras

import zipfile

In [9]:
print(tf.__version__)

2.2.0


In [4]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/vid_frame.zip", 'r')
zip_ref.extractall("/content/emotion")
zip_ref.close()

In [5]:
TRAINING_DIR = "/content/emotion/training/"
VALIDATION_DIR = "/content/emotion/validation/"

In [10]:
opt = keras.optimizers.Adam(learning_rate =0.001)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=30)
mc = ModelCheckpoint('/content/drive/My Drive/BestInceptionV3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

img_height = 224
img_width = 224
batch_size = 16
nb_epochs = 50

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)
# validation_split=0.1) # set validation split

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')
# subset='training') # set as training data

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = train_datagen.flow_from_directory(
    VALIDATION_DIR, # same directory as training data
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode='categorical')
# subset='validation') # set as validation data

Found 52314 images belonging to 8 classes.
Found 5550 images belonging to 8 classes.


In [12]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
emotions = Dense(8, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=emotions)

for layer in base_model.layers:
  layer.trainable = True

87916544/87910968 [==============================] - 1s 0us/step


In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

model.fit_generator(train_generator,
                    steps_per_epoch = train_generator.samples // batch_size,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples // batch_size,
                    epochs = nb_epochs, callbacks=[es, mc])

model.save('/content/drive/My Drive/InceptionV3')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
3269/3269 [==============================] - ETA: 0s - loss: 0.7366 - acc: 0.7428
Epoch 00001: val_acc improved from -inf to 0.32767, saving model to /content/drive/My Drive/BestInceptionV3.h5
3269/3269 [==============================] - 461s 141ms/step - loss: 0.7366 - acc: 0.7428 - val_loss: 2.7387 - val_acc: 0.3277
Epoch 2/50
3269/3269 [==============================] - ETA: 0s - loss: 0.3239 - acc: 0.8905
Epoch 00002: val_acc improved from 0.32767 to 0.53559, saving model to /content/drive/My Drive/BestInceptionV3.h5
3269/3269 [==============================] - 461s 141ms/step - loss: 0.3239 - acc: 0.8905 - val_loss: 2.0000 - val_acc: 0.5356
Epoch 3/50
3269/3269 [==============================] - ETA: 0s - loss: 0.2091 - acc: 0.9295
Epoch 00003: val_acc did not improve from 0.53559
3269/3269 [==============================] - 457s 140ms/step - loss: 0.2091 - acc: 0.9295 - val_loss: 2.5252 - val_a

In [15]:
read_model = keras.models.load_model("/content/drive/My Drive/BestInceptionV3.h5")

In [17]:
TESTING_DIR = "/content/emotion/testing/"

test_datagen = ImageDataGenerator(rescale=1./255)

testing_generator = test_datagen.flow_from_directory(
    TESTING_DIR, # same directory as training data
    target_size= (img_height, img_width),
    batch_size= batch_size,
    class_mode='categorical')

read_model.evaluate(testing_generator)

Found 5528 images belonging to 8 classes.
346/346 [==============================] - 14s 41ms/step - loss: 2.0715 - acc: 0.5013


[2.0714948177337646, 0.5012663006782532]